In [3]:
# Setting date and Portfolio as index
df.set_index(['date', 'Portfolio'], inplace=True)

# Function to plot time series for each portfolio
def plot_time_series(df, feature):
    plt.figure(figsize=(14, 7))
    portfolios = df.index.get_level_values('Portfolio').unique()

    for portfolio in portfolios:
        # Select data for the current portfolio and the specific feature
        ts_data = df.xs(portfolio, level='Portfolio')[feature]
        plt.plot(ts_data, label=portfolio, marker='o', linestyle='-')  # Marker to highlight each data point

    plt.title(f'Time Series Plot for {feature}')
    plt.xlabel('Date')
    plt.ylabel(feature)
    plt.legend(title='Portfolio')
    plt.grid(True)
    plt.show()

# Call the function with the feature you want to plot
plot_time_series(df, 'feature_1')

NameError: name 'plt' is not defined

In [3]:
import pandas as pd
import numpy as np

def aggregate_features(df):
    # Define the columns for which you want to calculate specific aggregations
    numerical_cols = ['feature1', 'feature2', 'feature3']  # replace with your actual feature columns
    binary_cols = ['binary_feature1', 'binary_feature2']  # replace with your actual binary feature columns
    
    # Define aggregation functions for numerical columns
    aggregations = {
        'min': 'min',
        'max': 'max',
        'mean': 'mean',
        'var': 'var',
        'q25': lambda x: np.percentile(x, 25),
        'q75': lambda x: np.percentile(x, 75),
        'iqr': lambda x: np.percentile(x, 75) - np.percentile(x, 25),
        'sum': 'sum'
    }

    # Prepare a dict to hold column-specific aggregations
    aggregations_dict = {}
    for col in numerical_cols:
        aggregations_dict[col] = aggregations
    
    for col in binary_cols:
        aggregations_dict[col] = {'sum': 'sum', 'mean': 'mean'}

    # Apply aggregations
    aggregated = df.groupby(['date', 'Portfolio']).agg(aggregations_dict)
    
    # Flatten the MultiIndex columns
    aggregated.columns = ['_'.join(col).strip() for col in aggregated.columns.values]

    return aggregated

# Example usage
# Assume X_train and X_test are already loaded DataFrames
aggregated_train = aggregate_features(X_train)
aggregated_test = aggregate_features(X_test)

# Optionally, save or display the aggregated data
print(aggregated_train.head())
print(aggregated_test.head())


SpecificationError: nested renamer is not supported